# Neural Style Transfer

Neural Style Transfer (NST) takes a style image $S$ (e.g. a painting) and applies its style to a content image $C$ to produce a new image $G$. Because a new image is generated, a model that performs NST is called a generative model.

<br>

<div style="text-align:center">
    <img src="media/nst.png">
    <caption><center><font color="purple">Examples of Neural Style Transfer</font></center></caption>
</div>

We can train such a model by training a NN that uses two cost functions:

**Notation:**
- C: content image
- S: style image
- G: generated image

$J_{\text{content}}(C,G)$: Cost regarding the content of the original content image $C$ and the generated image $G$ (content cost function).

$J_{\text{style}}(S,G)$: Cost regarding the style of the original style image $S$ and the generated image $G$ (style cost function)

Both cost function can be combined to a single cost function:

$$J(G) = \alpha \cdot J_{\text{content}}(C, G) + \beta \cdot J_{\text{style}}(S, G)$$

This cost function can be minimized the same way as in regular NN. To do this, the image $G$ is initialized with random noise and then optimized by applying gradient descent to minimize the costs.

## Content Cost

To understand how the content cost function works, we can visualize what a deep NN is learning by inspecting the activations of neurons in different layers. By visualizing the image patches that maximally activate a neuron, we can get a sense for what the neurons are learning. It turns out that the NN usually learns abstract things like "vertical edges", "bright/dark" etc. in middle-to-higher layers and more complex things like "water", "dogs" etc. in deeper layers:

<br>

<div style="text-align:center">
    <img src="media/nn_layers.png">
    <caption><center><font color="purple"><br>Activation Visualizations</font></center></caption>
</div>

We can calculate the content cost at any layer in the network and thus control how big its influence is on the generated image. Let's for example consider an image of a dog as the content image $C$. If we calculate the content cost in a higher layer we force the network to generate an image which looks similar to $C$. If we calculate the content cost in deeper layers we allow the network to generate an almost arbitrary image as long as there is a dog in the image. Usually some hidden layer in the middle is chosen to achieve a good balance between content and style.

Let $a^{[l](C)}$ and $a^{[l](G)}$ be the activations of layer $l$ for the content image $C$ and the style image $S$ respectively. The content cost function can now be defined as the element-wise square difference:

$$J^{[l]}_{\text{content}}(C, G) = \frac{1}{2} ||a^{[l](C)} - a^{[l](G)} ||^2 $$

## Style Cost
To calculate the similarity between the styles of two images, we can define style as the correlation between the activations across channels in a layer $l$. This correlation can be understood as follows:

- **High correlation:** An image patch which has a high value in both channel A and channel B contains a style property in both channels.
- **Low correlation:** An image patch which has a high value in channel A and a small value in channel B (or vice versa) contains style properties of channel A, but not the properties of channel B.

Let's visualize this with an example. Consider the two high-level style properties "contains vertical lines" and "has an orange tint" which are reflected in different channels. If the two properties are highly correlated it means the original style image $G$ often contains vertical lines in conjunction with an orange tint. We can therefore measure the similarity in style of the generated image $G$ by checking if the correlation between these properties (channels) is high too.

This can be expressed more formal as follows: Let $a^{[l]}_{i,j,k}$ be the activation of neuron $i$ in layer $l$ for the pixel at position $j,k$ in the style image $S$. We can represent the correlation between the $n_C$ channels in this layer as a style matrix $G$ (a.k.a. Gram-Matrix) which has the dimensions $(n_c \times n_c)$. This is easiest if the image matrix is unrolled into a 2-dimensional matrix:

<br>

<div style="text-align:center">
    <img src="media/nst-unrolling.png">
</div>

The elements of this matrix can then be calculated as follows:

$$G^{[l]}_{kk'} = \sum_{i=1}^{n^{[l]}_H} \sum_{j=1}^{n^{[l]}_W} a^{[l]}_{i, j, k} \cdot a^{[l]}_{i, j, k'}$$

<br>

<div style="text-align:center">
    <img src="media/gram-matrix.png">
</div>

This style matrix can be computed separately for both the style image $S$ and the generated image $G$. The SCF can then be defined as:

$$J^{[l]}_{\text{style}}(S,G) = ||G^{[l](S)} - G^{[l](G)}||^2_{F} = \frac{1}{s \cdot n^{[l]}_H \cdot n^{[l]}_W \cdot n^{[l]}_C} \cdot \sum_k \sum_{k'} \left( G_{kk'}^{[l](S)} - G_{kk'}^{[l](G)} \right)^2$$

The SCF can be applied on different layers (low-level and high-level layers) whereas the results can be weighted by appliying a parameter $\lambda^{[l]}$ and summed up to get the overall style cost across all layers:

$$J_{\text{style}}(S,G) = \sum_l \lambda^{[l]} \cdot J^{[l]}_{\text{style}}(S,G)$$